# SNEWS_PT Examples


## Publication

In [3]:
from SNEWS_PT.snews_pub import Publisher, SNEWSTiers

with Publisher() as pub:
    messages = SNEWSTiers(detector_name='KamLAND', timing_series=['4:31:05:565','4:31:05:575','4:31:05:585'], p_value=0.000007, machine_time='22/02/08 4:31:08:565',).determine_tier()
    pub.send(messages)

---------------------------------------------------------
_id                 :18_CoincidenceTier_21/12/16_17:11:53:875996
detector_name       :DS-20K
sent_time           :21/12/16 17:11:53
machine_time        :None
neutrino_time       :17:11:53:847107
p_value             :0.98


## Subscription

In [2]:
from SNEWS_PT.snews_sub import Subscriber

Subscriber().subscribe()


You are subscribing to ALERT
Broker:kafka://kafka.scimma.org/snews.alert-test
Done
